# Simple Perceptron Model with Homomorphic Encryption


# Build Perceptron (Without Encryption)

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

## Define the Perceptron Model
We start by defining the Perceptron class, which includes initializing weights, forward pass, and weight update methods.

In [2]:
class Perceptron:
    def __init__(self, learning_rate=0.1, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)
                update = self.lr * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted

    def _unit_step_func(self, x):
        return np.where(x > 0, 1, 0)


In [3]:
class OneVsRestClassifier:
    def __init__(self, classifier, *args, **kwargs):
        self.classifier = classifier
        self.args = args
        self.kwargs = kwargs
        self.classifiers = []

    def fit(self, X, y):
        self.unique_classes = np.unique(y)
        for i in self.unique_classes:
            y_binary = (y == i).astype(int)
            clf = self.classifier(*self.args, **self.kwargs)
            clf.fit(X, y_binary)
            self.classifiers.append(clf)
        return self

    def predict(self, X):
        predictions = np.array([clf.predict(X) for clf in self.classifiers]).T
        return np.argmax(predictions, axis=1)

## Load Iris Dataset

In [4]:
iris = datasets.load_iris()
x = iris.data
y = iris.target


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [17]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Train the Perceptron

In [18]:
p = OneVsRestClassifier(Perceptron,learning_rate=0.01, n_iters=100)
p.fit(x_train, y_train)


## Evaluate the Model
We can now test the trained model with the test dataset.

In [19]:
predictions = p.predict(x_test)
print("Model predictions:", predictions)
print("Actual labels:", y_test)


Model predictions: [1 0 2 0 1 0 0 2 1 0 2 0 0 0 0 0 2 1 1 2 0 2 0 2 2 2 1 2 0 0 0 0 0 0 0 2 0
 0]
Actual labels: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


In [20]:
# Calculate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 81.58%


# Build Perceptron (With Encryption)

In [21]:
pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 35.6 MB/s eta 0:00:00


In [70]:
# Generate a context for the CKKS scheme
import tenseal as ts
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()

In [71]:
class EncryptedPerceptron:
    def __init__(self, context, learning_rate=0.1, n_iters=1000, n_features=None):
        self.context = context
        self.lr = learning_rate
        self.n_iters = n_iters
        self.n_features = n_features
        self.activation_func = self._unit_step_func
        self.weights = np.zeros(n_features)  # Plaintext weights
        self.bias = 0  # Plaintext bias

    def fit(self, X, y):
        y_ = np.array([1 if i > 0 else 0 for i in y])
        for _ in range(self.n_iters):
            for idx, encrypted_x_i in enumerate(X):
                # Encrypted dot product with plaintext weights converted temporarily to encrypted vector
                enc_weights = ts.ckks_vector(self.context, self.weights.tolist())
                encrypted_linear_output = encrypted_x_i.dot(enc_weights) + self.bias

                # Decrypt only for activation to compare with label
                linear_output = encrypted_linear_output.decrypt()
                y_predicted = self.activation_func(linear_output)

                # Compute update in plaintext due to the need to use the result in non-linear functions and update rules
                update = self.lr * (y_[idx] - y_predicted)

                # Encrypt update to perform encrypted computation
                enc_update = ts.ckks_vector(self.context, [update] * self.n_features)  # replicate update to match feature vector size

                # Encrypted element-wise update
                enc_update_vector = encrypted_x_i * enc_update
                update_vector = enc_update_vector.decrypt()  # decrypt to update weights in plaintext

                # Update weights and bias directly in plaintext
                self.weights += update_vector
                self.bias += update

    def predict(self, X):
        results = []
        for encrypted_x_i in X:
            # Predict using encrypted input and plaintext weights converted to encrypted vector
            enc_weights = ts.ckks_vector(self.context, self.weights.tolist())
            encrypted_linear_output = encrypted_x_i.dot(enc_weights) + self.bias
            linear_output = encrypted_linear_output.decrypt()  # Decrypt to apply activation
            y_predicted = self.activation_func(linear_output)
            results.append(y_predicted)
        return results

    def _unit_step_func(self, x):
        return 1 if sum(x) > 0 else 0


In [72]:
class EncryptedOneVsRestClassifier:
    def __init__(self, context, classifier, learning_rate=0.1, n_iters=1000, n_features=None):
        self.context = context
        self.classifier = classifier
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.n_features = n_features
        self.classifiers = []

    def fit(self, X, y):
        self.unique_classes = np.unique(y)
        for i in self.unique_classes:
            y_binary = np.array([1 if label == i else 0 for label in y])
            clf = self.classifier(self.context, self.learning_rate, self.n_iters, self.n_features)
            clf.fit(X, y_binary)  # X is assumed to be already a list of encrypted vectors
            self.classifiers.append(clf)
        return self

    def predict(self, X):
        predictions = np.array([clf.predict(X) for clf in self.classifiers]).T
        return np.argmax(predictions, axis=1)

In [73]:
x_train_encrypted = [ts.ckks_vector(context, x) for x in x_train]
x_test_encrypted = [ts.ckks_vector(context, x) for x in x_test]

In [74]:
p = EncryptedOneVsRestClassifier(context, EncryptedPerceptron,learning_rate=0.1, n_iters=100, n_features=4)
p.fit(x_train_encrypted, y_train)
#Takes about 15 minutes to train

In [75]:
predictions = p.predict(x_test_encrypted)
print("Model predictions:", predictions)
print("Actual labels:", y_test)

Model predictions: [1 0 2 0 1 0 0 2 1 0 2 0 0 0 0 0 2 1 1 2 0 2 0 2 2 2 1 2 0 0 0 0 0 0 0 2 0
 0]
Actual labels: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


In [76]:
# Calculate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 81.58%
